<a href="https://colab.research.google.com/github/fahaerte/LSIES-FinalAssignment/blob/main/Task3_SVDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Execute the following cell to connect your google drive to the colab. Needed to load the data stored on your personal google drive. In case you use a local environment just comment out the following lines of code.

In [ ]:
# Environment variables
drive = True
install_packages = True

# If necessary, install packages
if install_packages:
  !pip install Basemap basemap-data-hires
  !pip install pandas matplotlib numpy
  !pip install sklearn
  !pip install datetime

# Imports
import os
import pandas as pd
import numpy as np
from csv import reader
from sklearn.metrics import r2_score
from datetime import datetime
from datetime import timedelta as tdelta
from datetime import time as time
import random
import warnings

# Mount drive if needed
if drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Colab\ Notebooks/LSIES

from Helper_funcs import *
from SensorPositions_Func import *
from SVD_Func import *

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Depending on the environemnt, set directory to data
if drive:
  dir_path = '/content/drive/MyDrive/data/'
else :
  dir_path = './data/'

folders = ['region_1_mustamäe_kristiine', 'region_2_data_kesklinn', 'region_3_kadriorg_lasnamäe', 'region_4_ülemiste']
csv_file_name_sensor_positions = 'sensor_positions.csv'
columns_sensors_positions = ['sensor_name', 'latitude', 'longitude', 'region']

In [3]:
#Import data
df_data_incomplete = import_sensor_data(dir_path)

#interpolate data using 'nearest' method
list_df_other = []
list_df_linear = []
list_df_nearest = []

temp_df = df_data_incomplete.interpolate(method='nearest')
df_data_nearest = temp_df.interpolate(limit_area=None, method='backfill')

temp_df = df_data_incomplete.interpolate(method='linear')
df_data_linear = temp_df.interpolate(limit_area=None, method='backfill')
df_data_linear.apply(lambda x: np.round(x, 1), 1);

In [4]:
# Compute indices of timestamps
day_index = compute_period_indices(df_data_linear, '07:00', '19:00')
evening_index = compute_period_indices(df_data_linear, '19:00', '23:00')
night_index = compute_period_indices(df_data_linear, '23:00', '7:00')

In [5]:
# Compute sensor regions
sensor_positions_df = import_sensor_positions(dir_path, csv_file_name_sensor_positions)
sensors_region1, sensors_region2, sensors_region3, sensors_region4 = get_sensors_based_on_region(sensor_positions_df)
del sensor_positions_df, csv_file_name_sensor_positions

In [ ]:
# Compute global SVDs
df_without_timestamp = df_data_linear.reset_index().iloc[:, 1:len(df_data_linear.columns)+1]
U_global, s_global, Vt_global = np.linalg.svd(df_without_timestamp)
del df_without_timestamp

In [ ]:
# Compute RMSE and SingVal based on time period for Global SVD
rmse_list_day, amount_sing_values_day = calc_RMSE_sinVal_based_on_time_period(U_global, s_global, Vt_global, df_data_linear, day_index)
rmse_list_evening, amount_sing_values_evening = calc_RMSE_sinVal_based_on_time_period(U_global, s_global, Vt_global, df_data_linear, evening_index)
rmse_list_night, amount_sing_values_night = calc_RMSE_sinVal_based_on_time_period(U_global, s_global, Vt_global, df_data_linear, night_index)

In [ ]:
# Plot RMSE and Singular Values of global SVD
plot_rmse_sinval(rmse_list_day, amount_sing_values_day, 'Global model - Day period')
plot_rmse_sinval(rmse_list_evening, amount_sing_values_evening, 'Global model - Evening period')
plot_rmse_sinval(rmse_list_night, amount_sing_values_night, 'Global model - Night period')

In [ ]:
# Collect garbage
del rmse_list_day, amount_sing_values_day, rmse_list_evening, amount_sing_values_evening, rmse_list_night, amount_sing_values_night
del temp_df, df_data_incomplete

In [ ]:
# Compute SVDs for region 1
U_reg1, s_reg1, Vt_reg1 = np.linalg.svd(df_data_linear[sensors_region1])

In [ ]:
# Region 1
rmse_list_day_reg1, amount_sing_values_day_reg1 = calc_RMSE_sinVal_based_on_time_period(U_reg1, s_reg1, Vt_reg1, df_data_linear[sensors_region1], day_index)
rmse_list_evening_reg1, amount_sing_values_evening_reg1 = calc_RMSE_sinVal_based_on_time_period(U_reg1, s_reg1, Vt_reg1, df_data_linear[sensors_region1], evening_index)
rmse_list_night_reg1, amount_sing_values_night_reg1 = calc_RMSE_sinVal_based_on_time_period(U_reg1, s_reg1, Vt_reg1, df_data_linear[sensors_region1], night_index)

# Plot region 1
plot_rmse_sinval(rmse_list_day_reg1, amount_sing_values_day_reg1, 'Region 1 model - Day period')
plot_rmse_sinval(rmse_list_evening_reg1, amount_sing_values_evening_reg1, 'Region 1 model - Evening period')
plot_rmse_sinval(rmse_list_night_reg1, amount_sing_values_night_reg1, 'Region 1 model - Night period')

del rmse_list_day_reg1, rmse_list_evening_reg1, rmse_list_night_reg1, amount_sing_values_day_reg1, amount_sing_values_evening_reg1, amount_sing_values_night_reg1

In [ ]:
# Compute SVDs for region 2
U_reg2, s_reg2, Vt_reg2 = np.linalg.svd(df_data_linear[sensors_region2])

In [ ]:
# Region 2
rmse_list_day_reg2, amount_sing_values_day_reg2 = calc_RMSE_sinVal_based_on_time_period(U_reg2, s_reg2, Vt_reg2, df_data_linear[sensors_region2], day_index)
rmse_list_evening_reg2, amount_sing_values_evening_reg2 = calc_RMSE_sinVal_based_on_time_period(U_reg2, s_reg2, Vt_reg2, df_data_linear[sensors_region2], evening_index)
rmse_list_night_reg2, amount_sing_values_night_reg2 = calc_RMSE_sinVal_based_on_time_period(U_reg2, s_reg2, Vt_reg2, df_data_linear[sensors_region2], night_index)

# Plot region 2
plot_rmse_sinval(rmse_list_day_reg2, amount_sing_values_day_reg2, 'Region 2 model - Day period')
plot_rmse_sinval(rmse_list_evening_reg2, amount_sing_values_evening_reg2, 'Region 2 model - Evening period')
plot_rmse_sinval(rmse_list_night_reg2, amount_sing_values_night_reg2, 'Region 2 model - Night period')

del rmse_list_day_reg2, rmse_list_evening_reg2, rmse_list_night_reg2, amount_sing_values_day_reg2, amount_sing_values_evening_reg2, amount_sing_values_night_reg2

In [ ]:
# Compute SVDs for region 3
U_reg3, s_reg3, Vt_reg3 = np.linalg.svd(df_data_linear[sensors_region3])

In [ ]:
# Region 3
rmse_list_day_reg3, amount_sing_values_day_reg3 = calc_RMSE_sinVal_based_on_time_period(U_reg3, s_reg3, Vt_reg3, df_data_linear[sensors_region3], day_index)
rmse_list_evening_reg3, amount_sing_values_evening_reg3 = calc_RMSE_sinVal_based_on_time_period(U_reg3, s_reg3, Vt_reg3, df_data_linear[sensors_region3], evening_index)
rmse_list_night_reg3, amount_sing_values_night_reg3 = calc_RMSE_sinVal_based_on_time_period(U_reg3, s_reg3, Vt_reg3, df_data_linear[sensors_region3], night_index)

# Plot region 3
plot_rmse_sinval(rmse_list_day_reg3, amount_sing_values_day_reg3, 'Region 3 model - Day period')
plot_rmse_sinval(rmse_list_evening_reg3, amount_sing_values_evening_reg3, 'Region 3 model - Evening period')
plot_rmse_sinval(rmse_list_night_reg3, amount_sing_values_night_reg3, 'Region 3 model - Night period')

del rmse_list_day_reg3, rmse_list_evening_reg3, rmse_list_night_reg3, amount_sing_values_day_reg3, amount_sing_values_evening_reg3, amount_sing_values_night_reg3

In [ ]:
# Compute SVDs for region 4
U_reg4, s_reg4, Vt_reg4 = np.linalg.svd(df_data_linear[sensors_region4])

In [ ]:
# Region 4
rmse_list_day_reg4, amount_sing_values_day_reg4 = calc_RMSE_sinVal_based_on_time_period(U_reg4, s_reg4, Vt_reg4, df_data_linear[sensors_region4], day_index)
rmse_list_evening_reg4, amount_sing_values_evening_reg4 = calc_RMSE_sinVal_based_on_time_period(U_reg4, s_reg4, Vt_reg4, df_data_linear[sensors_region4], evening_index)
rmse_list_night_reg4, amount_sing_values_night_reg4 = calc_RMSE_sinVal_based_on_time_period(U_reg4, s_reg4, Vt_reg4, df_data_linear[sensors_region4], night_index)

# Plot region 4
plot_rmse_sinval(rmse_list_day_reg4, amount_sing_values_day_reg4, 'Region 4 model - Day period')
plot_rmse_sinval(rmse_list_evening_reg4, amount_sing_values_evening_reg4, 'Region 4 model - Evening period')
plot_rmse_sinval(rmse_list_night_reg4, amount_sing_values_night_reg4, 'Region 4 model - Night period')

del rmse_list_day_reg4, rmse_list_evening_reg4, rmse_list_night_reg4, amount_sing_values_day_reg4, amount_sing_values_evening_reg4, amount_sing_values_night_reg4

In [6]:
sampling_sizes = [0.2, 0.3, 0.4, 0.5]
start_time = datetime.strptime('2022.08.01 00:00:00', '%Y.%m.%d %H:%M:%S')
end_time = datetime.strptime('2022.08.13 23:59:00', '%Y.%m.%d %H:%M:%S')

samples = [random_walk_uni(int(df_data_linear.shape[0] * s), 0, 1) for s in sampling_sizes]
sample_dfs = []

time_diff = end_time - start_time

# Scale the random walk values to the time range
for sample in samples:
  time_values = start_time + (time_diff * sample)
  time_values = [timestamp.replace(second=0).replace(microsecond=0) for timestamp in time_values]
  sample_dfs.append(pd.DataFrame({"Timestamp": time_values}))

In [7]:
chosen_sensors = ['237B', '23B1','22CC', '2329']
for df in sample_dfs:
  df['Timestamp'] = pd.to_datetime(df['Timestamp'])
  df = df.merge(df_data_linear[chosen_sensors], how='inner', left_on = 'Timestamp', right_index=True)

In [8]:
# Pick 4 stations and apply the different SVD models
stat_list = [
    {'id': '237B', 'region': 1, 'real_data': df_data_linear[chosen_sensors[0]]},
    {'id': '23B1', 'region': 2, 'real_data': df_data_linear[chosen_sensors[1]]},
    {'id': '22CC', 'region': 3, 'real_data': df_data_linear[chosen_sensors[2]]},
    {'id': '2329', 'region': 4, 'real_data': df_data_linear[chosen_sensors[3]]}
]

# Compute global SVDs
df_without_timestamp = df_data_linear.reset_index().iloc[:, 1:len(df_data_linear.columns)+1]
U_global, s_global, Vt_global = np.linalg.svd(df_without_timestamp)
s_global = keep_low_rank_model_of_svd(df=df_data_linear, s=s_global, rank=110) # RMSE approx 1.5 with a rank of 110

for stat in stat_list:
  stat['recon_global'] = apply_svd(df_data_linear, stat['id'], U_global, s_global, Vt_global)
  # stat = compare_results(stat, 'comp_to_global', stat['real_data'], stat['recon_global'])

del df_without_timestamp, U_global, s_global, Vt_global

# Compute SVDs for region 1
U_reg1, s_reg1, Vt_reg1 = np.linalg.svd(df_data_linear[sensors_region1])
s_reg1 = keep_low_rank_model_of_svd(df=df_data_linear[sensors_region1], s=s_reg1, rank=40) # RMSE approx 1 with a rank of 40
stat_list[0]['recon_reg'] = apply_svd(df_data_linear[sensors_region1], stat_list[0]['id'], U_reg1, s_reg1, Vt_reg1)
# stat_list[0] = compare_results(stat_list[0], 'comp_to_recon_reg', stat_list[0]['real_data'], stat_list[0]['recon_reg'])

del U_reg1, s_reg1, Vt_reg1

# Compute SVDs for region 2
U_reg2, s_reg2, Vt_reg2 = np.linalg.svd(df_data_linear[sensors_region2])
s_reg2 = keep_low_rank_model_of_svd(df=df_data_linear[sensors_region2], s=s_reg2, rank=83) # RMSE approx 1 with a rank of 83
stat_list[1]['recon_reg'] = apply_svd(df_data_linear[sensors_region2], stat_list[1]['id'], U_reg2, s_reg2, Vt_reg2)
# stat_list[1] = compare_results(stat_list[1], 'comp_to_recon_reg', stat_list[1]['real_data'], stat_list[1]['recon_reg'])

del U_reg2, s_reg2, Vt_reg2

# Compute SVDs for region 3
U_reg3, s_reg3, Vt_reg3 = np.linalg.svd(df_data_linear[sensors_region3])
s_reg3 = keep_low_rank_model_of_svd(df=df_data_linear[sensors_region3], s=s_reg3, rank=38) # RMSE approx 1 with a rank of 35
stat_list[2]['recon_reg'] = apply_svd(df_data_linear[sensors_region3], stat_list[2]['id'], U_reg3, s_reg3, Vt_reg3)
# stat_list[2] = compare_results(stat_list[2], 'comp_to_recon_reg', stat_list[2]['real_data'], stat_list[2]['recon_reg'])

del U_reg3, s_reg3, Vt_reg3

# Compute SVDs for region 4
U_reg4, s_reg4, Vt_reg4 = np.linalg.svd(df_data_linear[sensors_region4])
s_reg4 = keep_low_rank_model_of_svd(df=df_data_linear[sensors_region4], s=s_reg4, rank=6) # RMSE approx 1 with a rank of 6
stat_list[3]['recon_reg'] = apply_svd(df_data_linear[sensors_region4], stat_list[3]['id'], U_reg4, s_reg4, Vt_reg4)
# stat_list[3] = compare_results(stat_list[3], 'comp_to_recon_reg', stat_list[3]['real_data'], stat_list[3]['recon_reg'])

del U_reg4, s_reg4, Vt_reg4

In [11]:
# Print values
for stat in stat_list:
  sensor_df = pd.DataFrame(stat['real_data'])
  sensor_df['recon_global'] = stat['recon_global']
  sensor_df['recon_reg'] = stat['recon_reg']

  for sample_df, size in zip(sample_dfs, sampling_sizes):
    sample_df = sample_df.merge(sensor_df, how='inner', left_on = 'Timestamp', right_index=True)
    glob_rmse, glob_mad = compare_results(sample_df[stat['id']], sample_df['recon_global'])
    reg_rmse, reg_mad = compare_results(sample_df[stat['id']], sample_df['recon_reg'])
    fstring = f"SensorId, {stat['id']} in region {stat['region']} sampling size {size}: RMSE (global): {round(glob_rmse, 3)} MAD (global): {round(glob_mad, 3)} RMSE (region): {round(reg_rmse, 3)} MAD (region): {round(reg_mad, 3)}"
    print(fstring)
  
  print('--------------------------')

# TODO: Plot data

SensorId, 237B in region 1 sampling size 0.2: RMSE (global): 1.568 MAD (global): 4.315 RMSE (region): 0.728 MAD (region): 4.313
SensorId, 237B in region 1 sampling size 0.3: RMSE (global): 1.579 MAD (global): 4.297 RMSE (region): 0.715 MAD (region): 4.307
SensorId, 237B in region 1 sampling size 0.4: RMSE (global): 1.572 MAD (global): 4.399 RMSE (region): 0.716 MAD (region): 4.405
SensorId, 237B in region 1 sampling size 0.5: RMSE (global): 1.556 MAD (global): 4.312 RMSE (region): 0.715 MAD (region): 4.319
--------------------------
SensorId, 23B1 in region 2 sampling size 0.2: RMSE (global): 1.213 MAD (global): 3.472 RMSE (region): 0.674 MAD (region): 3.453
SensorId, 23B1 in region 2 sampling size 0.3: RMSE (global): 1.203 MAD (global): 3.4 RMSE (region): 0.665 MAD (region): 3.42
SensorId, 23B1 in region 2 sampling size 0.4: RMSE (global): 1.206 MAD (global): 3.518 RMSE (region): 0.662 MAD (region): 3.518
SensorId, 23B1 in region 2 sampling size 0.5: RMSE (global): 1.22 MAD (global): 